## Loading the data 

In [1]:
# import the neccesary libtraries
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")



In [2]:
districts_info = pd.read_csv("D:/10 Academy/Week8/districts_info.csv")
products_info = pd.read_csv("D:/10 Academy/products_info.csv")

## District data View

In [3]:
districts_info.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
districts_info.shape

(233, 7)

In [5]:
# a look at the percentage of the  missing data
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The district dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(districts_info)

The district dataset contains 27.1 % missing values.


In [6]:
# columns and the missing values
 
districts_info.isna().sum()

district_id                   0
state                        57
locale                       57
pct_black/hispanic           57
pct_free/reduced             85
county_connections_ratio     71
pp_total_raw                115
dtype: int64

From the data we can observe that there are a lot of missing values with teh highest percentage fo teh missing values from the one in the 
free/ reduced column. Therefore we are going to look at a way we can make good inference at the end with this missing values. 

dropping the row for states that are not very ompactful in the data engimeering

In [7]:
#dropping nulls states 
districts_info = districts_info[districts_info.state.notna()].reset_index(drop=True)

In [8]:
districts_info.isna().sum()

district_id                  0
state                        0
locale                       0
pct_black/hispanic           0
pct_free/reduced            28
county_connections_ratio    14
pp_total_raw                58
dtype: int64

In [9]:
# fill missing with ffill method for columns 
# Forward fill. 

def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]


def fix_missing_bfill(df, col):
    df[col] = df[col].fillna(method='bfill')
    return df[col]

districts_info['pct_free/reduced'] = fix_missing_ffill(districts_info, 'pct_free/reduced')
districts_info['county_connections_ratio'] = fix_missing_ffill(districts_info, 'county_connections_ratio')
districts_info['pp_total_raw'] = fix_missing_ffill(districts_info, 'pp_total_raw')


In [10]:
districts_info.isna().sum()


district_id                 0
state                       0
locale                      0
pct_black/hispanic          0
pct_free/reduced            0
county_connections_ratio    0
pp_total_raw                0
dtype: int64

In [11]:
for i in ['pct_black/hispanic', 'pct_free/reduced']:
    districts_info[i] = districts_info[i].apply(lambda x: float(x.split(',')[0][1:]) + 0.1)

districts_info['pp_total_raw'] = districts_info['pp_total_raw'].apply(lambda x: int(x.split(',')[0][1:]) + 1000)

districts_info.drop('county_connections_ratio', axis = 1, inplace = True)

districts_info.head(10)

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,8815,Illinois,Suburb,0.1,0.1,15000
1,4921,Utah,Suburb,0.1,0.3,7000
2,5987,Wisconsin,Suburb,0.1,0.1,11000
3,3710,Utah,Suburb,0.1,0.5,7000
4,7177,North Carolina,Suburb,0.3,0.3,9000
5,9812,Utah,Suburb,0.1,0.3,7000
6,6584,North Carolina,Rural,0.5,0.7,9000
7,1044,Missouri,Suburb,0.1,0.1,11000
8,7457,Washington,City,0.5,0.5,13000
9,1904,Connecticut,Rural,0.1,0.1,13000


In [12]:
products_info.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Si..."
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


## Engagement data 

In [32]:
PATH = 'D:/10 Academy/Week8/engagement_data' 

temp = []

for dist in districts_info.district_id.unique():
    df = pd.read_csv(f'{PATH}/{dist}.csv', index_col=None, header=0)
    df["district_id"] = dist
    temp.append(df)
    
    
engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)
engagement.head(5)

,time,lp_id,pct_access,engagement_index,district_id
0,2020-01-27,32213.0,100.00,3000.00,8815
1,2020-02-25,90153.0,33.33,2666.67,8815
2,2020-02-25,99916.0,0.00,NaN,8815
3,2020-02-25,28504.0,0.00,NaN,8815
4,2020-02-25,95731.0,33.33,333.33,8815


In [33]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17435744 entries, 0 to 17435743
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   time              object 
 1   lp_id             float64
 2   pct_access        float64
 3   engagement_index  float64
 4   district_id       int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 665.1+ MB


In [34]:
engagement.isna().sum()

time                      0
lp_id                   399
pct_access             7362
engagement_index    4297347
district_id               0
dtype: int64

In [35]:
# handling the empty data in the dataset

engagement['time'] = pd.to_datetime(engagement['time'])
        
    # Add time features

    # Holidays
engagement["holiday"] = np.where(engagement["time"].isin(["2020-01-30", "2020-01-31"]), 1,
        np.where((engagement["time"]>="2020-04-05") & (engagement["time"]<="2020-04-09"), 1,
        np.where((engagement["time"]>="2020-06-12") & (engagement["time"]<="2020-09-10"), 1,
        np.where((engagement["time"]>="2020-11-25") & (engagement["time"]<="2020-11-29"), 1,
        np.where((engagement["time"]>="2020-12-21") & (engagement["time"]<="2020-12-31"), 1, 0)))))

    # Before/After Outbreak
engagement["outbreak"] = np.where(engagement["time"]<="2020-03-24", 0, 1)

In [36]:
engagement['month']= pd.to_datetime(engagement['time']).dt.month_name()
engagement['weekday']= pd.to_datetime(engagement['time']).dt.day_name()

In [37]:

#we can make the month and weekday into numeric if thats agreed on 
engagement

,time,lp_id,pct_access,engagement_index,district_id,holiday,outbreak,month,weekday
0,2020-01-27,32213.0,100.00,3000.00,8815,0,0,January,Monday
1,2020-02-25,90153.0,33.33,2666.67,8815,0,0,February,Tuesday
2,2020-02-25,99916.0,0.00,NaN,8815,0,0,February,Tuesday
3,2020-02-25,28504.0,0.00,NaN,8815,0,0,February,Tuesday
4,2020-02-25,95731.0,33.33,333.33,8815,0,0,February,Tuesday
...,...,...,...,...,...,...,...,...,...
17435739,2020-12-31,43009.0,0.02,8.14,7164,1,1,December,Thursday
17435740,2020-12-31,62984.0,0.00,0.04,7164,1,1,December,Thursday
17435741,2020-12-31,90014.0,0.00,0.04,7164,1,1,December,Thursday
17435742,2020-12-31,61945.0,0.29,16.06,7164,1,1,December,Thursday


Look at the engagement

In [13]:
engagement_dict = {
    "time":	"date in \"YYYY-MM-DD\"",
    "lp_id" :	"The unique identifier of the product",
    "pct_access" :	"Percentage of students in the district have at least one page-load event of a given product and on a given day",
    "engagement_index"	: "Total page-load events per one thousand students of a given product and on a given day"
}
pd.DataFrame.from_dict(engagement_dict, orient="index", columns=["description"])

,description
time,"date in ""YYYY-MM-DD"""
lp_id,The unique identifier of the product
pct_access,Percentage of students in the district have at...
engagement_index,Total page-load events per one thousand studen...


A look at the district Data

In [14]:
pd.set_option('display.max_colwidth', None)  
districts_info_desc = {
    "district_id"	: "The unique identifier of the school district",
    "state" :	"The state where the district resides in",
    "locale" :	"NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.",
    "pct_black/hispanic" :	"Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data",
    "pct_free/reduced" : 	"Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data",
    "countyconnectionsratio" :	"ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.",
    "pptotalraw" :	"Per-pupil total expenditupd.set_option('display.max_rows', 500)re (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district."
}
pd.DataFrame.from_dict(districts_info_desc, orient="index", columns=["description"])

,description
district_id,The unique identifier of the school district
state,The state where the district resides in
locale,"NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information."
pct_black/hispanic,Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data
pct_free/reduced,Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data
countyconnectionsratio,ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.
pptotalraw,"Per-pupil total expenditupd.set_option('display.max_rows', 500)re (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district."


## Product Information 

A look at the product information

In [15]:
products_info_desc = {
    "LP ID"	: "The unique identifier of the product",
    "URL":	"Web Link to the specific product",
"Product Name" :	"Name of the specific product",
"Provider/Company Name" :	"Name of the product provider",
"Sector(s)"	: "Sector of education where the product is used",
"Primary Essential Function" :	"The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled"
}
pd.DataFrame.from_dict(products_info_desc, orient="index", columns=["description"])

,description
LP ID,The unique identifier of the product
URL,Web Link to the specific product
Product Name,Name of the specific product
Provider/Company Name,Name of the product provider
Sector(s),Sector of education where the product is used
Primary Essential Function,"The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled"


In [16]:
products_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LP ID                       372 non-null    int64 
 1   URL                         372 non-null    object
 2   Product Name                372 non-null    object
 3   Provider/Company Name       371 non-null    object
 4   Sector(s)                   352 non-null    object
 5   Primary Essential Function  352 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ KB


In [17]:
products_info.isna().sum()

LP ID                          0
URL                            0
Product Name                   0
Provider/Company Name          1
Sector(s)                     20
Primary Essential Function    20
dtype: int64

We seperate the primary essential for a better understatnding of the data 

In [18]:
products_info['Sector(s)'] = fix_missing_ffill(products_info, 'Sector(s)')
products_info['Primary Essential Function'] = fix_missing_ffill(products_info, 'Primary Essential Function')
# Use of the ffill
products_info['primary_function_main'] = products_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
products_info['primary_function_sub'] = products_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

products_info['primary_function_sub'] = products_info['primary_function_sub'].replace({'Sites, Resources & References' : 'Sites, Resources & Reference'})
products_info.drop("Primary Essential Function", axis=1, inplace=True)


products_info

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC,Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,LC,"Sites, Resources & Reference"
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC,Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC,Digital Learning Platforms
...,...,...,...,...,...,...,...
367,88065,https://dochub.com/,DocHub,DocHub,PreK-12; Higher Ed; Corporate,SDO,Other
368,37805,http://google.com/slides/about/,Google Slides,Google LLC,PreK-12; Higher Ed; Corporate,LC,Content Creation & Curation
369,32555,http://www.innersloth.com/gameAmongUs.php,Among Us,InnerSloth,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"
370,87841,http://edpuzzle.com,Edpuzzle - Free (Basic Plan),EDpuzzle Inc.,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"


In [19]:
products_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    1
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [20]:
products_info = products_info.dropna()

In [21]:
products_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    0
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [41]:
merged_data = pd.merge(products_info, engagement, left_on = 'LP ID', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, districts_info, on = 'district_id')
#merged_data.drop(['URL', 'lp_id'], axis = 1, inplace = True)
merged_data.head(3)

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub,time,lp_id,pct_access,...,district_id,holiday,outbreak,month,weekday,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-02-27,13117.0,0.14,...,8815,0,0,February,Thursday,Illinois,Suburb,0.1,0.1,15000
1,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-02,13117.0,0.18,...,8815,0,0,March,Monday,Illinois,Suburb,0.1,0.1,15000
2,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-03,13117.0,0.08,...,8815,0,0,March,Tuesday,Illinois,Suburb,0.1,0.1,15000


In [43]:
merged_data.drop(['URL', 'lp_id'], axis = 1, inplace = True)

In [44]:
merged_data.shape

(9139701, 19)

In [45]:
merged_data.head()

,LP ID,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub,time,pct_access,engagement_index,district_id,holiday,outbreak,month,weekday,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-02-27,0.14,2.84,8815,0,0,February,Thursday,Illinois,Suburb,0.1,0.1,15000
1,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-02,0.18,6.13,8815,0,0,March,Monday,Illinois,Suburb,0.1,0.1,15000
2,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-03,0.08,3.18,8815,0,0,March,Tuesday,Illinois,Suburb,0.1,0.1,15000
3,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-04,0.14,8.02,8815,0,0,March,Wednesday,Illinois,Suburb,0.1,0.1,15000
4,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-05,0.39,69.37,8815,0,0,March,Thursday,Illinois,Suburb,0.1,0.1,15000
